In [1]:
# Importing the necessary Libraries
import pandas as pd
import numpy as np
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.max_colwidth',None)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from datetime import timedelta


import warnings 
warnings.filterwarnings('ignore')

In [2]:
data=pd.read_excel('../data/file.xlsx')
print(data.shape)
data.head()

(2090, 13)


,User_id,Session_id,DateTime,Category,SubCategory,Action,Quantity,Rate,Total Price,Recency,Frequency,Monetary,Clusters
0,52243841613,d76fde-8bb3-4e00-8c23,2019-01-10 10:20:00,Electronic Appliances,Speakers,first_app_open,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,52243841613,33dfbd-b87a-4708-9857,2019-01-10 10:22:00,Electronic Appliances,Speakers,search,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,57314161118,6511c2-e2e3-422b-b695,2019-01-10 14:00:00,Men's Fashion,Jeans,search,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,57314161118,90fc70-0e80-4590-96f3,2019-01-10 14:07:00,Men's Fashion,Jeans,product_view,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,57314161118,bd7419-2748-4c56-95b4,2019-01-10 14:12:00,Men's Fashion,Jeans,read_reviews,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2090 entries, 0 to 2089
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   User_id      2090 non-null   int64         
 1   Session_id   2090 non-null   object        
 2   DateTime     2090 non-null   datetime64[ns]
 3   Category     2090 non-null   object        
 4   SubCategory  2090 non-null   object        
 5   Action       2090 non-null   object        
 6   Quantity     201 non-null    float64       
 7   Rate         201 non-null    float64       
 8   Total Price  201 non-null    float64       
 9   Recency      1303 non-null   float64       
 10  Frequency    1303 non-null   float64       
 11  Monetary     1303 non-null   float64       
 12  Clusters     1303 non-null   float64       
dtypes: datetime64[ns](1), float64(7), int64(1), object(4)
memory usage: 212.4+ KB


In [4]:
data.Action.unique()

array(['first_app_open', 'search', 'product_view', 'read_reviews',
       'add_review', 'add_to_cart', 'checkout', 'purchase',
       'add_to_wishlist', 'click_wishlist_page', 'product_review',
       'add_to_wishist', 'read_review'], dtype=object)

In [5]:
print(len(data.Action.value_counts()))
data['Action'] = data['Action'].apply(lambda x: 'read_reviews' if x in ('read_review') else x)
data['Action'] = data['Action'].apply(lambda x: 'add_to_wishlist' if x in ('add_to_wishist') else x)
print(len(data.Action.value_counts()))

13
11


In [6]:
# Creating date level features
data['Date'] = data['DateTime'].dt.date
data['DayOfWeek'] = data['DateTime'].dt.dayofweek
data['DayOfMonth'] = data['DateTime'].dt.day

In [7]:
data.head()

,User_id,Session_id,DateTime,Category,SubCategory,Action,Quantity,Rate,Total Price,Recency,Frequency,Monetary,Clusters,Date,DayOfWeek,DayOfMonth
0,52243841613,d76fde-8bb3-4e00-8c23,2019-01-10 10:20:00,Electronic Appliances,Speakers,first_app_open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10
1,52243841613,33dfbd-b87a-4708-9857,2019-01-10 10:22:00,Electronic Appliances,Speakers,search,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10
2,57314161118,6511c2-e2e3-422b-b695,2019-01-10 14:00:00,Men's Fashion,Jeans,search,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10
3,57314161118,90fc70-0e80-4590-96f3,2019-01-10 14:07:00,Men's Fashion,Jeans,product_view,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10
4,57314161118,bd7419-2748-4c56-95b4,2019-01-10 14:12:00,Men's Fashion,Jeans,read_reviews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10


## Features:

### no of active days:

In [8]:
temp_days_active = data.groupby('User_id').agg({'Date': 'nunique'}).reset_index()
temp_days_active.rename(columns={'Date':'no_of_days_active'},inplace=True)
print(temp_days_active.shape)
temp_days_active.head(3)

(401, 2)


,User_id,no_of_days_active
0,51443820326,1
1,51448162784,3
2,51455512201,3


In [9]:
# Adding days_active
data = pd.merge(data, temp_days_active, on='User_id', how='left')
print(data.shape)
data.head(3)

(2090, 17)


,User_id,Session_id,DateTime,Category,SubCategory,Action,Quantity,Rate,Total Price,Recency,Frequency,Monetary,Clusters,Date,DayOfWeek,DayOfMonth,no_of_days_active
0,52243841613,d76fde-8bb3-4e00-8c23,2019-01-10 10:20:00,Electronic Appliances,Speakers,first_app_open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10,1
1,52243841613,33dfbd-b87a-4708-9857,2019-01-10 10:22:00,Electronic Appliances,Speakers,search,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10,1
2,57314161118,6511c2-e2e3-422b-b695,2019-01-10 14:00:00,Men's Fashion,Jeans,search,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10,1


In [10]:
df=data

### avrg time:

In [11]:
temp = df.copy()

In [12]:
def purch_time_diff(x):
    if len(x) == 2:
        return (x[1]-x[0]).days

    if len(x) == 3:
        diff_12 = (x[1]-x[0]).days
        diff_23 = (x[2]-x[1]).days
        return (diff_12+diff_23)/2

In [13]:
temp_user_purchase_date = (temp[temp['Action']=='purchase'].sort_values(by='Date')
                           .groupby('User_id').agg({'Date': lambda x: list(x)}).reset_index())
temp_user_purchase_date['purchase1'] = temp_user_purchase_date['Date'].apply(lambda x: x[0])
temp_user_purchase_date['purchase2'] = temp_user_purchase_date['Date'].apply(lambda x: x[1] if len(x)>1 else 0)
temp_user_purchase_date['purchase3'] = temp_user_purchase_date['Date'].apply(lambda x: x[2] if len(x)>2 else 0)
temp_user_purchase_date['avg_time_between_purchase'] = temp_user_purchase_date['Date'].apply(purch_time_diff)

print(temp_user_purchase_date.shape)
temp_user_purchase_date.head()

(167, 6)


,User_id,Date,purchase1,purchase2,purchase3,avg_time_between_purchase
0,51448162784,"[2019-06-11, 2019-12-12]",2019-06-11,2019-12-12,0,184.0
1,51455512201,[2019-10-19],2019-10-19,0,0,NaN
2,51462598385,"[2019-10-20, 2019-12-15]",2019-10-20,2019-12-15,0,56.0
3,51463622150,"[2019-10-14, 2019-11-12]",2019-10-14,2019-11-12,0,29.0
4,51465529647,"[2019-12-05, 2019-12-12]",2019-12-05,2019-12-12,0,7.0


### cart to purchase ratio(users)

In [14]:
del temp

In [15]:
temp = df.copy()

In [16]:
def purchase_ratios(action_col='add_to_cart',id_col='User_id', col_name='no_of_carts'):
    # Number of actions for each user
    test1 = (temp[temp['Action']==action_col]
                          .groupby(id_col).agg({'Session_id':'count'}).reset_index())
    test1.rename(columns={'Session_id': col_name},inplace=True)
    print(test1.shape)
    
    # Number of purchases for each user
    test2 = temp[temp['Action']=='purchase'].groupby(id_col).agg({'Session_id':'count'}).reset_index()
    test2.rename(columns={'Session_id':'no_of_purch'},inplace=True)
    print(test2.shape)  
    
    test_ratio = pd.merge(test1,test2,on=id_col, how='left')
    test_ratio = test_ratio.fillna(0)
    test_ratio[col_name+'_to_purchase_ratio'] = (test_ratio['no_of_purch']
                                                       /test_ratio[col_name])
    print(test_ratio.shape)
    return test_ratio


In [17]:
# cart to purchase ratio
temp_cart_purch_ratio_user = purchase_ratios(action_col='add_to_cart',id_col='User_id',col_name='no_of_carts')
print(temp_cart_purch_ratio_user.shape)
display(temp_cart_purch_ratio_user.head())
temp_cart_purch_ratio_user.describe()

(240, 2)
(167, 2)
(240, 4)
(240, 4)


,User_id,no_of_carts,no_of_purch,no_of_carts_to_purchase_ratio
0,51448162784,2,2.0,1.000000
1,51455512201,1,1.0,1.000000
2,51462598385,2,2.0,1.000000
3,51463622150,2,2.0,1.000000
4,51465529647,3,2.0,0.666667


,User_id,no_of_carts,no_of_purch,no_of_carts_to_purchase_ratio
count,2.400000e+02,240.000000,240.000000,240.000000
mean,5.396592e+10,1.166667,0.754167,0.614583
std,2.767010e+09,0.405678,0.698162,0.534009
min,5.144816e+10,1.000000,0.000000,0.000000
25%,5.185721e+10,1.000000,0.000000,0.000000
50%,5.231850e+10,1.000000,1.000000,1.000000
75%,5.586744e+10,1.000000,1.000000,1.000000
max,6.030264e+10,3.000000,3.000000,3.000000


In [18]:
import gc

In [19]:
gc.collect()

109

### Wishlist to Purchase Ratio:

In [20]:
del temp

In [21]:
temp = df.copy()

In [22]:
# Add wishlist to purchase ratio
temp_add_wishlist_purch_ratio_user = purchase_ratios(action_col='add_to_wishlist',
                                             id_col='User_id',col_name='no_of_add_wishlist')
print(temp_add_wishlist_purch_ratio_user.shape)
display(temp_add_wishlist_purch_ratio_user.head())
temp_add_wishlist_purch_ratio_user.describe()

(26, 2)
(167, 2)
(26, 4)
(26, 4)


,User_id,no_of_add_wishlist,no_of_purch,no_of_add_wishlist_to_purchase_ratio
0,51463622150,1,2.0,2.00
1,51477448981,2,1.0,0.50
2,51517997197,2,2.0,1.00
3,51526464449,1,2.0,2.00
4,51565158157,4,3.0,0.75


,User_id,no_of_add_wishlist,no_of_purch,no_of_add_wishlist_to_purchase_ratio
count,2.600000e+01,26.000000,26.000000,26.000000
mean,5.295457e+10,1.461538,1.538462,1.214744
std,2.529052e+09,0.760567,0.706018,0.694491
min,5.146362e+10,1.000000,0.000000,0.000000
25%,5.161642e+10,1.000000,1.000000,0.812500
50%,5.175573e+10,1.000000,2.000000,1.000000
75%,5.230051e+10,2.000000,2.000000,2.000000
max,5.953342e+10,4.000000,3.000000,2.000000


In [23]:
# Add wishlist to purchase ratio
temp_add_wishlist_purch_ratio_user = purchase_ratios(action_col='add_to_wishlist',
                                             id_col='User_id',col_name='no_of_add_wishlist')
print(temp_add_wishlist_purch_ratio_user.shape)
display(temp_add_wishlist_purch_ratio_user.head())
temp_add_wishlist_purch_ratio_user.describe()

(26, 2)
(167, 2)
(26, 4)
(26, 4)


,User_id,no_of_add_wishlist,no_of_purch,no_of_add_wishlist_to_purchase_ratio
0,51463622150,1,2.0,2.00
1,51477448981,2,1.0,0.50
2,51517997197,2,2.0,1.00
3,51526464449,1,2.0,2.00
4,51565158157,4,3.0,0.75


,User_id,no_of_add_wishlist,no_of_purch,no_of_add_wishlist_to_purchase_ratio
count,2.600000e+01,26.000000,26.000000,26.000000
mean,5.295457e+10,1.461538,1.538462,1.214744
std,2.529052e+09,0.760567,0.706018,0.694491
min,5.146362e+10,1.000000,0.000000,0.000000
25%,5.161642e+10,1.000000,1.000000,0.812500
50%,5.175573e+10,1.000000,2.000000,1.000000
75%,5.230051e+10,2.000000,2.000000,2.000000
max,5.953342e+10,4.000000,3.000000,2.000000


### cart to purchase ratio -categories

In [24]:
del temp

In [25]:
temp = df.copy()

In [26]:
temp.head(3)

,User_id,Session_id,DateTime,Category,SubCategory,Action,Quantity,Rate,Total Price,Recency,Frequency,Monetary,Clusters,Date,DayOfWeek,DayOfMonth,no_of_days_active
0,52243841613,d76fde-8bb3-4e00-8c23,2019-01-10 10:20:00,Electronic Appliances,Speakers,first_app_open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10,1
1,52243841613,33dfbd-b87a-4708-9857,2019-01-10 10:22:00,Electronic Appliances,Speakers,search,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10,1
2,57314161118,6511c2-e2e3-422b-b695,2019-01-10 14:00:00,Men's Fashion,Jeans,search,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10,1


In [27]:
# Category level - cart_to_purchase_ratio
temp_cart_purch_ratio_category = purchase_ratios(action_col='add_to_cart',
                                                 id_col='Category',col_name='category_no_of_carts')
display(temp_cart_purch_ratio_category.head())
temp_cart_purch_ratio_category.describe()

(89, 2)
(75, 2)
(89, 4)


,Category,category_no_of_carts,no_of_purch,category_no_of_carts_to_purchase_ratio
0,Accessories,15,12.0,0.8
1,Air Conditioner,1,2.0,2.0
2,Appliances,1,1.0,1.0
3,Artwork,1,0.0,0.0
4,Audio,1,0.0,0.0


,category_no_of_carts,no_of_purch,category_no_of_carts_to_purchase_ratio
count,89.000000,89.000000,89.000000
mean,3.146067,2.224719,0.741005
std,3.540514,2.708374,0.564977
min,1.000000,0.000000,0.000000
25%,1.000000,1.000000,0.333333
50%,2.000000,1.000000,0.750000
75%,3.000000,3.000000,1.000000
max,16.000000,14.000000,3.000000


In [28]:
# SubCategory level - cart_to_purchase_ratio
temp_cart_purch_ratio_subcategory = (purchase_ratios(action_col='add_to_cart',
                                                     id_col='SubCategory',col_name='subcategory_no_of_carts'))
display(temp_cart_purch_ratio_subcategory.head())
temp_cart_purch_ratio_subcategory.describe()

(239, 2)
(176, 2)
(239, 4)


,SubCategory,subcategory_no_of_carts,no_of_purch,subcategory_no_of_carts_to_purchase_ratio
0,Air Tight Container,1,0.0,0.0
1,Air coolers,1,1.0,1.0
2,Amazon Alexa,1,0.0,0.0
3,Amplifier,1,1.0,1.0
4,Audio Speakers,1,0.0,0.0


,subcategory_no_of_carts,no_of_purch,subcategory_no_of_carts_to_purchase_ratio
count,239.000000,239.000000,239.000000
mean,1.171548,0.769874,0.656555
std,0.654546,0.711086,0.499926
min,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000
max,8.000000,4.000000,2.000000


In [29]:
gc.collect()

0

### product to purchase ratio

In [30]:
del temp

In [31]:
temp = df.copy()

In [32]:
# Category level - produc_view_wishlist_to_purchase_ratio
temp_product_view_purch_ratio_category = (purchase_ratios(action_col='product_view',
                                                        id_col='Category',col_name='category_no_of_product_view'))
print(temp_product_view_purch_ratio_category.shape)
display(temp_product_view_purch_ratio_category.head())
temp_product_view_purch_ratio_category.describe()

(90, 2)
(75, 2)
(90, 4)
(90, 4)


,Category,category_no_of_product_view,no_of_purch,category_no_of_product_view_to_purchase_ratio
0,Accessories,9,12.0,1.333333
1,Air Conditioner,7,2.0,0.285714
2,Artwork,1,0.0,0.000000
3,Audio,1,0.0,0.000000
4,Baby Care,3,1.0,0.333333


,category_no_of_product_view,no_of_purch,category_no_of_product_view_to_purchase_ratio
count,90.000000,90.000000,90.000000
mean,3.244444,2.077778,0.607009
std,3.468352,2.769123,0.634233
min,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,2.000000,1.000000,0.500000
75%,4.000000,2.750000,1.000000
max,26.000000,14.000000,3.000000


In [33]:
# SubCategory level - produc_view_wishlist_to_purchase_ratio
temp_product_view_purch_ratio_subcategory = (purchase_ratios(action_col='product_view',
                                                    id_col='SubCategory',col_name='subcategory_no_of_product_view'))
print(temp_product_view_purch_ratio_subcategory.shape)
display(temp_product_view_purch_ratio_subcategory.head())
temp_product_view_purch_ratio_subcategory.describe()

(216, 2)
(176, 2)
(216, 4)
(216, 4)


,SubCategory,subcategory_no_of_product_view,no_of_purch,subcategory_no_of_product_view_to_purchase_ratio
0,Amazon Alexa,1,0.0,0.000000
1,Arm sleeve,1,0.0,0.000000
2,Aux Cable,3,1.0,0.333333
3,Baby Shampoo,1,0.0,0.000000
4,Baby Shower gel,1,0.0,0.000000


,subcategory_no_of_product_view,no_of_purch,subcategory_no_of_product_view_to_purchase_ratio
count,216.000000,216.000000,216.000000
mean,1.351852,0.495370,0.319422
std,0.860663,0.753273,0.431870
min,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000
75%,1.000000,1.000000,0.666667
max,7.000000,4.000000,1.500000


## modelling

In [34]:
# Target group of people are those who added items to the cart (Trigger point)
df_base = (df[df['Action']=='add_to_cart']
              .groupby('User_id').agg({'Category':'max','SubCategory':'max'}).reset_index())

print(df_base.shape)
df_base.head()

(240, 3)


,User_id,Category,SubCategory
0,51448162784,Camera Accessories,Camera Lens
1,51455512201,Men's Fashion,Shoes
2,51462598385,Mobile & Accessories,USB Cord
3,51463622150,Stationary,Paints
4,51465529647,Home Decor,Wall hangings


In [35]:
# Adding days_active
df_base = pd.merge(df_base, temp_days_active, on='User_id', how='left')
print(df_base.shape)
df_base.head(3)

(240, 4)


,User_id,Category,SubCategory,no_of_days_active
0,51448162784,Camera Accessories,Camera Lens,3
1,51455512201,Men's Fashion,Shoes,3
2,51462598385,Mobile & Accessories,USB Cord,5


In [37]:
data.head()

,User_id,Session_id,DateTime,Category,SubCategory,Action,Quantity,Rate,Total Price,Recency,Frequency,Monetary,Clusters,Date,DayOfWeek,DayOfMonth,no_of_days_active
0,52243841613,d76fde-8bb3-4e00-8c23,2019-01-10 10:20:00,Electronic Appliances,Speakers,first_app_open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10,1
1,52243841613,33dfbd-b87a-4708-9857,2019-01-10 10:22:00,Electronic Appliances,Speakers,search,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10,1
2,57314161118,6511c2-e2e3-422b-b695,2019-01-10 14:00:00,Men's Fashion,Jeans,search,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10,1
3,57314161118,90fc70-0e80-4590-96f3,2019-01-10 14:07:00,Men's Fashion,Jeans,product_view,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10,1
4,57314161118,bd7419-2748-4c56-95b4,2019-01-10 14:12:00,Men's Fashion,Jeans,read_reviews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-10,3,10,1


In [38]:
# Adding RFM features
temp_rfm_feats = df.groupby('User_id').agg({'Recency': 'max','Frequency':'max','Monetary':'max','Clusters':'max'}).reset_index()
df_base = pd.merge(df_base, temp_rfm_feats, on='User_id', how='left')
print(df_base.shape)
df_base.head(3)

(240, 8)


,User_id,Category,SubCategory,no_of_days_active,Recency,Frequency,Monetary,Clusters
0,51448162784,Camera Accessories,Camera Lens,3,10.0,2.0,21500.0,3.0
1,51455512201,Men's Fashion,Shoes,3,64.0,1.0,10000.0,2.0
2,51462598385,Mobile & Accessories,USB Cord,5,7.0,2.0,550.0,3.0


In [39]:
# Adding average time between purchase
df_base = pd.merge(df_base, temp_user_purchase_date[['User_id','avg_time_between_purchase']], 
                   on='User_id', how='left')
print(df_base.shape)
df_base.head(3)

(240, 9)


,User_id,Category,SubCategory,no_of_days_active,Recency,Frequency,Monetary,Clusters,avg_time_between_purchase
0,51448162784,Camera Accessories,Camera Lens,3,10.0,2.0,21500.0,3.0,184.0
1,51455512201,Men's Fashion,Shoes,3,64.0,1.0,10000.0,2.0,NaN
2,51462598385,Mobile & Accessories,USB Cord,5,7.0,2.0,550.0,3.0,56.0


In [40]:
# Adding carts to purchase ratio
df_base = pd.merge(df_base, temp_cart_purch_ratio_user[['User_id','no_of_carts_to_purchase_ratio']], 
                   on='User_id', how='left')
print(df_base.shape)
df_base.head(3)

(240, 10)


,User_id,Category,SubCategory,no_of_days_active,Recency,Frequency,Monetary,Clusters,avg_time_between_purchase,no_of_carts_to_purchase_ratio
0,51448162784,Camera Accessories,Camera Lens,3,10.0,2.0,21500.0,3.0,184.0,1.0
1,51455512201,Men's Fashion,Shoes,3,64.0,1.0,10000.0,2.0,NaN,1.0
2,51462598385,Mobile & Accessories,USB Cord,5,7.0,2.0,550.0,3.0,56.0,1.0


In [43]:
# Adding wishlist to purchase ratios (click & add)
#not_added
# df_base = pd.merge(df_base, 
#                    temp_click_wishlist_purch_ratio_user[['User_id','no_of_click_wishlist_to_purchase_ratio']], 
#                    on='User_id', how='left')
# print(df_base.shape)

df_base = pd.merge(df_base, 
                   temp_add_wishlist_purch_ratio_user[['User_id','no_of_add_wishlist_to_purchase_ratio']], 
                   on='User_id', how='left')
print(df_base.shape)

df_base.head(3)

(240, 13)


,User_id,Category,SubCategory,no_of_days_active,Recency,Frequency,Monetary,Clusters,avg_time_between_purchase,no_of_carts_to_purchase_ratio,category_no_of_carts_to_purchase_ratio,subcategory_no_of_carts_to_purchase_ratio,no_of_add_wishlist_to_purchase_ratio
0,51448162784,Camera Accessories,Camera Lens,3,10.0,2.0,21500.0,3.0,184.0,1.0,0.300000,1.0,NaN
1,51455512201,Men's Fashion,Shoes,3,64.0,1.0,10000.0,2.0,NaN,1.0,0.800000,1.0,NaN
2,51462598385,Mobile & Accessories,USB Cord,5,7.0,2.0,550.0,3.0,56.0,1.0,0.933333,1.0,NaN


In [42]:
# Adding Category & SubCategory level carts-to-purchase ratio
df_base = pd.merge(df_base, temp_cart_purch_ratio_category[['Category','category_no_of_carts_to_purchase_ratio']], 
                   on='Category', how='left')

df_base = pd.merge(df_base,
                   temp_cart_purch_ratio_subcategory[['SubCategory','subcategory_no_of_carts_to_purchase_ratio']], 
                   on='SubCategory', how='left')

print(df_base.shape)
df_base.head(3)

(240, 12)


,User_id,Category,SubCategory,no_of_days_active,Recency,Frequency,Monetary,Clusters,avg_time_between_purchase,no_of_carts_to_purchase_ratio,category_no_of_carts_to_purchase_ratio,subcategory_no_of_carts_to_purchase_ratio
0,51448162784,Camera Accessories,Camera Lens,3,10.0,2.0,21500.0,3.0,184.0,1.0,0.300000,1.0
1,51455512201,Men's Fashion,Shoes,3,64.0,1.0,10000.0,2.0,NaN,1.0,0.800000,1.0
2,51462598385,Mobile & Accessories,USB Cord,5,7.0,2.0,550.0,3.0,56.0,1.0,0.933333,1.0


In [45]:
# Adding Category & SubCategory level carts-to-purchase ratio
df_base = pd.merge(df_base, temp_cart_purch_ratio_category[['Category','category_no_of_carts_to_purchase_ratio']], 
                   on='Category', how='left')

df_base = pd.merge(df_base,
                   temp_cart_purch_ratio_subcategory[['SubCategory','subcategory_no_of_carts_to_purchase_ratio']], 
                   on='SubCategory', how='left')

print(df_base.shape)
df_base.head(3)

(240, 15)


,User_id,Category,SubCategory,no_of_days_active,Recency,Frequency,Monetary,Clusters,avg_time_between_purchase,no_of_carts_to_purchase_ratio,category_no_of_carts_to_purchase_ratio_x,subcategory_no_of_carts_to_purchase_ratio_x,no_of_add_wishlist_to_purchase_ratio,category_no_of_carts_to_purchase_ratio_y,subcategory_no_of_carts_to_purchase_ratio_y
0,51448162784,Camera Accessories,Camera Lens,3,10.0,2.0,21500.0,3.0,184.0,1.0,0.300000,1.0,NaN,0.300000,1.0
1,51455512201,Men's Fashion,Shoes,3,64.0,1.0,10000.0,2.0,NaN,1.0,0.800000,1.0,NaN,0.800000,1.0
2,51462598385,Mobile & Accessories,USB Cord,5,7.0,2.0,550.0,3.0,56.0,1.0,0.933333,1.0,NaN,0.933333,1.0


In [46]:
# Adding Category & SubCategory level product_view-to-purchase ratio (click & add)
df_base = pd.merge(df_base, 
                   temp_product_view_purch_ratio_category[['Category','category_no_of_product_view_to_purchase_ratio']], 
                   on='Category', how='left')

df_base = pd.merge(df_base,
                   temp_product_view_purch_ratio_subcategory[['SubCategory','subcategory_no_of_product_view_to_purchase_ratio']], 
                   on='SubCategory', how='left')

print(df_base.shape)
df_base.head(3)

(240, 17)


,User_id,Category,SubCategory,no_of_days_active,Recency,Frequency,Monetary,Clusters,avg_time_between_purchase,no_of_carts_to_purchase_ratio,category_no_of_carts_to_purchase_ratio_x,subcategory_no_of_carts_to_purchase_ratio_x,no_of_add_wishlist_to_purchase_ratio,category_no_of_carts_to_purchase_ratio_y,subcategory_no_of_carts_to_purchase_ratio_y,category_no_of_product_view_to_purchase_ratio,subcategory_no_of_product_view_to_purchase_ratio
0,51448162784,Camera Accessories,Camera Lens,3,10.0,2.0,21500.0,3.0,184.0,1.0,0.300000,1.0,NaN,0.300000,1.0,0.600000,1.0
1,51455512201,Men's Fashion,Shoes,3,64.0,1.0,10000.0,2.0,NaN,1.0,0.800000,1.0,NaN,0.800000,1.0,0.615385,0.5
2,51462598385,Mobile & Accessories,USB Cord,5,7.0,2.0,550.0,3.0,56.0,1.0,0.933333,1.0,NaN,0.933333,1.0,0.538462,1.0


In [47]:
# Adding the target variable
# Filtering the dataset by max purchase date for each user (All users who have done add_to_cart event)

# drop the Target column if already created
# df_base = df_base.drop('Target',axis=1)

temp = (df.sort_values(by='DateTime')
        [(df.DateTime == df.User_id.map(df[df['Action']=='purchase'].groupby('User_id').DateTime.max()))]
       )

print(temp.shape)
# df_base.head()

purchase_users = temp[temp['Action']=='purchase'].groupby('User_id')['Action'].count().reset_index()
display(purchase_users.head(5))

df_base = pd.merge(df_base, purchase_users,on='User_id',how='left')
df_base['Action'] = df_base['Action'].fillna(0)
df_base.rename(columns={'Action':'Target'},inplace=True)
print(df_base.shape)
print(df_base.Target.value_counts())
df_base.head()

(179, 17)


,User_id,Action
0,51448162784,1
1,51455512201,1
2,51462598385,1
3,51463622150,1
4,51465529647,1


(240, 18)
Target
1.0    147
0.0     93
Name: count, dtype: int64


,User_id,Category,SubCategory,no_of_days_active,Recency,Frequency,Monetary,Clusters,avg_time_between_purchase,no_of_carts_to_purchase_ratio,category_no_of_carts_to_purchase_ratio_x,subcategory_no_of_carts_to_purchase_ratio_x,no_of_add_wishlist_to_purchase_ratio,category_no_of_carts_to_purchase_ratio_y,subcategory_no_of_carts_to_purchase_ratio_y,category_no_of_product_view_to_purchase_ratio,subcategory_no_of_product_view_to_purchase_ratio,Target
0,51448162784,Camera Accessories,Camera Lens,3,10.0,2.0,21500.0,3.0,184.0,1.000000,0.300000,1.0,NaN,0.300000,1.0,0.600000,1.0,1.0
1,51455512201,Men's Fashion,Shoes,3,64.0,1.0,10000.0,2.0,NaN,1.000000,0.800000,1.0,NaN,0.800000,1.0,0.615385,0.5,1.0
2,51462598385,Mobile & Accessories,USB Cord,5,7.0,2.0,550.0,3.0,56.0,1.000000,0.933333,1.0,NaN,0.933333,1.0,0.538462,1.0,1.0
3,51463622150,Stationary,Paints,3,40.0,2.0,950.0,3.0,29.0,1.000000,0.642857,1.0,2.0,0.642857,1.0,1.285714,1.0,1.0
4,51465529647,Home Decor,Wall hangings,5,10.0,2.0,15500.0,3.0,7.0,0.666667,0.571429,1.0,NaN,0.571429,1.0,0.800000,NaN,1.0


In [48]:
df_base.shape

(240, 18)

In [49]:
df_base.isna().sum()

User_id                                               0
Category                                              0
SubCategory                                           0
no_of_days_active                                     0
Recency                                              93
Frequency                                            93
Monetary                                             93
Clusters                                             93
avg_time_between_purchase                           208
no_of_carts_to_purchase_ratio                         0
category_no_of_carts_to_purchase_ratio_x              0
subcategory_no_of_carts_to_purchase_ratio_x           0
no_of_add_wishlist_to_purchase_ratio                217
category_no_of_carts_to_purchase_ratio_y              0
subcategory_no_of_carts_to_purchase_ratio_y           0
category_no_of_product_view_to_purchase_ratio        16
subcategory_no_of_product_view_to_purchase_ratio    108
Target                                          

In [50]:
# Null value imputation
df_base['avg_time_between_purchase'] = df_base['avg_time_between_purchase'].fillna(-1)
df_base = df_base.fillna(0)
df_base.isna().sum()

User_id                                             0
Category                                            0
SubCategory                                         0
no_of_days_active                                   0
Recency                                             0
Frequency                                           0
Monetary                                            0
Clusters                                            0
avg_time_between_purchase                           0
no_of_carts_to_purchase_ratio                       0
category_no_of_carts_to_purchase_ratio_x            0
subcategory_no_of_carts_to_purchase_ratio_x         0
no_of_add_wishlist_to_purchase_ratio                0
category_no_of_carts_to_purchase_ratio_y            0
subcategory_no_of_carts_to_purchase_ratio_y         0
category_no_of_product_view_to_purchase_ratio       0
subcategory_no_of_product_view_to_purchase_ratio    0
Target                                              0
dtype: int64

In [51]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [52]:
df_model = df_base.copy()
print(df_model.shape)
df_model.head(3)

(240, 18)


,User_id,Category,SubCategory,no_of_days_active,Recency,Frequency,Monetary,Clusters,avg_time_between_purchase,no_of_carts_to_purchase_ratio,category_no_of_carts_to_purchase_ratio_x,subcategory_no_of_carts_to_purchase_ratio_x,no_of_add_wishlist_to_purchase_ratio,category_no_of_carts_to_purchase_ratio_y,subcategory_no_of_carts_to_purchase_ratio_y,category_no_of_product_view_to_purchase_ratio,subcategory_no_of_product_view_to_purchase_ratio,Target
0,51448162784,Camera Accessories,Camera Lens,3,10.0,2.0,21500.0,3.0,184.0,1.0,0.300000,1.0,0.0,0.300000,1.0,0.600000,1.0,1.0
1,51455512201,Men's Fashion,Shoes,3,64.0,1.0,10000.0,2.0,-1.0,1.0,0.800000,1.0,0.0,0.800000,1.0,0.615385,0.5,1.0
2,51462598385,Mobile & Accessories,USB Cord,5,7.0,2.0,550.0,3.0,56.0,1.0,0.933333,1.0,0.0,0.933333,1.0,0.538462,1.0,1.0


In [53]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 18 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   User_id                                           240 non-null    int64  
 1   Category                                          240 non-null    object 
 2   SubCategory                                       240 non-null    object 
 3   no_of_days_active                                 240 non-null    int64  
 4   Recency                                           240 non-null    float64
 5   Frequency                                         240 non-null    float64
 6   Monetary                                          240 non-null    float64
 7   Clusters                                          240 non-null    float64
 8   avg_time_between_purchase                         240 non-null    float64
 9   no_of_carts_to_purcha

In [55]:
# One-hot-Encoding
# cat_cols = df_model.select_dtypes(['object']).columns.tolist() # All categorical feats
 # Only top_paths feature
num_cols = df_model.drop(['Target','User_id'],axis=1).select_dtypes(['int','float']).columns.tolist()

# cat_df = df_model[cat_cols]
# cat_df = pd.get_dummies(cat_df)

# Scaling
scaler = MinMaxScaler()
num_df = scaler.fit_transform(df_model[num_cols])
num_df = pd.DataFrame(num_df, columns = num_cols)

final_df = pd.concat([num_df, df_model[['Target']]],axis=1)
print(final_df.shape)
final_df.head(3)

(240, 15)


,no_of_days_active,Recency,Frequency,Monetary,Clusters,avg_time_between_purchase,no_of_carts_to_purchase_ratio,category_no_of_carts_to_purchase_ratio_x,subcategory_no_of_carts_to_purchase_ratio_x,no_of_add_wishlist_to_purchase_ratio,category_no_of_carts_to_purchase_ratio_y,subcategory_no_of_carts_to_purchase_ratio_y,category_no_of_product_view_to_purchase_ratio,subcategory_no_of_product_view_to_purchase_ratio,Target
0,0.285714,0.028986,0.666667,0.134375,0.75,0.748988,0.333333,0.100000,0.5,0.0,0.100000,0.5,0.200000,0.666667,1.0
1,0.285714,0.185507,0.333333,0.062500,0.50,0.000000,0.333333,0.266667,0.5,0.0,0.266667,0.5,0.205128,0.333333,1.0
2,0.571429,0.020290,0.666667,0.003438,0.75,0.230769,0.333333,0.311111,0.5,0.0,0.311111,0.5,0.179487,0.666667,1.0


In [56]:
# Train & Test split
X = final_df.drop(['Target'],axis=1)
Y = final_df[['Target']]
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.3)
print((x_train.shape,y_train.shape), (x_test.shape,y_test.shape))

((168, 14), (168, 1)) ((72, 14), (72, 1))


In [57]:
print(y_train.value_counts())
print(y_train.value_counts(normalize=True))

Target
1.0       104
0.0        64
Name: count, dtype: int64
Target
1.0       0.619048
0.0       0.380952
Name: proportion, dtype: float64


In [58]:
# Baseline model
logreg = LogisticRegression(class_weight='balanced',random_state=123)
logreg.fit(x_train,y_train)

LogisticRegression(class_weight='balanced', random_state=123)

In [59]:
preds = logreg.predict_proba(x_test)[:,1]
print(preds)
print("Test ROC-AUC:" + str(metrics.roc_auc_score(y_test,preds)))

[0.13504889 0.95965815 0.82871982 0.98178233 0.06126899 0.99102541
 0.05252008 0.98105366 0.98206217 0.96044136 0.0650974  0.07132842
 0.07132842 0.16346761 0.0651935  0.93875306 0.99406157 0.06836267
 0.1135078  0.96158201 0.93980543 0.91253911 0.16197705 0.94178624
 0.92370527 0.96363275 0.92363777 0.05252008 0.9895349  0.91769009
 0.06784003 0.0651935  0.98121512 0.96630441 0.95307322 0.16346761
 0.91519988 0.1135078  0.93673239 0.94287809 0.0651935  0.94416907
 0.97267274 0.99196135 0.93366549 0.95128328 0.98944287 0.07239078
 0.05252008 0.06836267 0.91578868 0.06126899 0.94081251 0.06126899
 0.08376178 0.9325273  0.9322947  0.90476543 0.90329181 0.75842355
 0.06245822 0.99049015 0.92680083 0.9485194  0.07573044 0.17589306
 0.9910437  0.96883693 0.07510825 0.07200466 0.89903459 0.07723132]
Test ROC-AUC:1.0


In [71]:
def sigmoid(x):
  res= 1/(1+np.exp(-x))
  return res

In [75]:
res=sigmoid(preds)
res

array([0.533711  , 0.72305336, 0.69608417, 0.72746173, 0.51531246,
       0.72929041, 0.513127  , 0.72731724, 0.7275172 , 0.72321016,
       0.51626861, 0.51782455, 0.51782455, 0.54077614, 0.5162926 ,
       0.71884771, 0.72988941, 0.51708402, 0.52834652, 0.72343844,
       0.71906035, 0.71351946, 0.54040596, 0.71946033, 0.71579648,
       0.72384855, 0.71578275, 0.513127  , 0.72899605, 0.71457121,
       0.51695351, 0.5162926 , 0.72734926, 0.72438228, 0.72173281,
       0.54077614, 0.71406304, 0.52834652, 0.71843914, 0.71968065,
       0.5162926 , 0.71994102, 0.72565191, 0.72947515, 0.71781834,
       0.72137318, 0.72897787, 0.5180898 , 0.513127  , 0.51708402,
       0.71418324, 0.51531246, 0.71926375, 0.51531246, 0.52092821,
       0.71758774, 0.7175406 , 0.71192781, 0.7116255 , 0.68101137,
       0.51560948, 0.72918472, 0.71642579, 0.72081732, 0.51892357,
       0.54386024, 0.72929402, 0.72488761, 0.51876824, 0.51799339,
       0.71075107, 0.51929824])

In [78]:
for i in preds:
  print(1 if i >= 0.5 else 0)


0
1
1
1
0
1
0
1
1
1
0
0
0
0
0
1
1
0
0
1
1
1
0
1
1
1
1
0
1
1
0
0
1
1
1
0
1
0
1
1
0
1
1
1
1
1
1
0
0
0
1
0
1
0
0
1
1
1
1
1
0
1
1
1
0
0
1
1
0
0
1
0


In [64]:
x_test[:2]

,no_of_days_active,Recency,Frequency,Monetary,Clusters,avg_time_between_purchase,no_of_carts_to_purchase_ratio,category_no_of_carts_to_purchase_ratio_x,subcategory_no_of_carts_to_purchase_ratio_x,no_of_add_wishlist_to_purchase_ratio,category_no_of_carts_to_purchase_ratio_y,subcategory_no_of_carts_to_purchase_ratio_y,category_no_of_product_view_to_purchase_ratio,subcategory_no_of_product_view_to_purchase_ratio
24,0.000000,0.000000,0.000000,0.0000,0.00,0.0,0.000000,0.111111,0.25,0.0,0.111111,0.25,0.055556,0.000000
46,0.571429,0.965217,0.333333,0.0175,0.25,0.0,0.333333,0.083333,0.50,0.0,0.083333,0.50,0.111111,0.333333


In [65]:
x_test.count()

no_of_days_active                                   72
Recency                                             72
Frequency                                           72
Monetary                                            72
Clusters                                            72
avg_time_between_purchase                           72
no_of_carts_to_purchase_ratio                       72
category_no_of_carts_to_purchase_ratio_x            72
subcategory_no_of_carts_to_purchase_ratio_x         72
no_of_add_wishlist_to_purchase_ratio                72
category_no_of_carts_to_purchase_ratio_y            72
subcategory_no_of_carts_to_purchase_ratio_y         72
category_no_of_product_view_to_purchase_ratio       72
subcategory_no_of_product_view_to_purchase_ratio    72
dtype: int64

In [60]:
preds_label = logreg.predict(x_test)
print(preds_label)
print("Test Accuracy:" + str(metrics.accuracy_score(y_test,preds_label)))

[0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1.
 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0.
 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0.]
Test Accuracy:1.0


In [66]:
len(preds_label)

72

In [68]:
from sklearn.metrics import confusion_matrix





# Create confusion matrix
cm = confusion_matrix(y_test, preds_label)

print("Confusion Matrix:")
cm


Confusion Matrix:


array([[29,  0],
       [ 0, 43]], dtype=int64)

In [61]:
pd.DataFrame(zip(x_train.columns,logreg.coef_[0]), columns=['Feats','Imp']).sort_values(by='Imp',ascending=False)

,Feats,Imp
4,Clusters,2.925211
2,Frequency,2.161250
1,Recency,1.870991
8,subcategory_no_of_carts_to_purchase_ratio_x,1.796549
11,subcategory_no_of_carts_to_purchase_ratio_y,1.796549
6,no_of_carts_to_purchase_ratio,1.746191
7,category_no_of_carts_to_purchase_ratio_x,0.551540
10,category_no_of_carts_to_purchase_ratio_y,0.551540
0,no_of_days_active,0.517743
12,category_no_of_product_view_to_purchase_ratio,0.265234
